In [0]:
pip install msal

In [0]:
pip install adal

In [0]:
dbutils.library.restartPython() 

In [0]:
from datetime import datetime, time, timedelta, date
import itertools
from pyspark.sql.functions import last_day
from datta_pipeline_library.core.base_config import (
    BaseConfig,
    CollibraConfig,
    CommonConfig,
    EnvConfig
)
from datta_pipeline_library.helpers.adls import configure_spark_to_use_spn_to_write_to_adls_gen2
from datta_pipeline_library.helpers.spn import AzureSPN
# from datta_pipeline_library.edc.collibra import fetch_business_metadata
from datta_pipeline_library.helpers.uc import (
    get_catalog_name,
    get_raw_schema_name
)

In [0]:
env = dbutils.widgets.get(name="env")
repos_path = dbutils.widgets.get(name="Tech_EH_Repo_Path")
unique_repo_branch_id = dbutils.widgets.get(name="unique_repo_branch_id")
unique_repo_branch_id_schema = dbutils.widgets.get(name="unique_repo_branch_id_schema")
unique_repo_branch_id = unique_repo_branch_id.replace("datta","")
unique_repo_branch_id_schema = unique_repo_branch_id_schema.replace("datta","")
print("unique_repo_branch_id:",unique_repo_branch_id)
print("unique_repo_branch_id_schema:",unique_repo_branch_id_schema)

# env = "dev"
# repos_path = "/Repos/DATTA-AUTO_TRIGGER_CAPABILITY/DATTA-TECH-EH"
# unique_repo_branch_id = ""
# unique_repo_branch_id_schema = ""

common_conf = CommonConfig.from_file(f"/Workspace/{repos_path.strip('/')}/conf/common/common_conf.json")
print("common_conf !!!")
env_conf = EnvConfig.from_file(f"/Workspace/{repos_path.strip('/')}/conf/{env}/conf.json")

base_config = BaseConfig.from_confs(env_conf, common_conf)
if unique_repo_branch_id:
    base_config.set_unique_id(unique_repo_branch_id)
if unique_repo_branch_id_schema:
    base_config.set_unique_id_schema(unique_repo_branch_id_schema)

In [0]:
uc_catalog_name = base_config.get_uc_catalog_name()
print("uc_catalog_name : ",uc_catalog_name)
uc_raw_schema = base_config.get_uc_raw_schema()
print("uc_raw_schema : ",uc_raw_schema)

raw_folder_path = base_config.get_raw_folder_path()
print("raw_folder_path : ",raw_folder_path)

tbl_owner_grp = base_config.get_tbl_owner_grp()
print("tbl_owner_grp : ",tbl_owner_grp)
tbl_read_grp = base_config.get_tbl_read_grp()
print("tbl_read_grp : ",tbl_owner_grp)

In [0]:
num_hours = 1   #int, 
starttime = spark.sql(f"select cast(concat(date_format(dateadd(HOUR, 1, current_timestamp()),'HH'),':00:00') as timestamp)").first()[0] #timestamp
endtime  = spark.sql(f"select cast(concat(date_format(dateadd(HOUR, 2, current_timestamp()),'HH'),':00:00') as timestamp)").first()[0] #timestamp

ist_starttime = spark.sql(f"SELECT cast(concat(date_format(dateadd(HOUR, 1, from_utc_timestamp(current_timestamp(),'GMT+5:30')),'HH'),':00:00') as timestamp)").first()[0]
ist_endtime = spark.sql(f"SELECT cast(concat(date_format(dateadd(HOUR, 2, from_utc_timestamp(current_timestamp(),'GMT+5:30')),'HH'),':00:00') as timestamp)").first()[0]

if endtime == datetime.combine(date.today(),datetime.strptime('000000','%H%M%S').time()):
    endtime = datetime.combine(date.today(),datetime.strptime('235900','%H%M%S').time())
if ist_endtime == datetime.combine(date.today(),datetime.strptime('000000','%H%M%S').time()):
    ist_endtime = datetime.combine(date.today(),datetime.strptime('235900','%H%M%S').time())

print(starttime)
print(endtime)
print(ist_starttime)
print(ist_endtime)

In [0]:
__15_minutes_list = []
__30_minutes_list = []
__45_minutes_list = []
__60_minutes_list = []

In [0]:
def doset(k_list, map=map, list=list, set=set, tuple=tuple):
  return map(list, set(map(tuple, k_list)))

def recursive_auto_schedule_15_mins(end_time_15):
    if end_time_15 < endtime and endtime != datetime.combine(date.today(),datetime.strptime('000000','%H%M%S').time()):
        __15_minutes_list.append([end_time_15,(end_time_15 + timedelta(minutes=15)), float('15')])
        return recursive_auto_schedule_15_mins(end_time_15 + timedelta(minutes=15))
    else:
        return __15_minutes_list

def recursive_auto_schedule_30_mins(end_time_30):
    if end_time_30 < endtime and endtime != datetime.combine(date.today(),datetime.strptime('000000','%H%M%S').time()):
        __30_minutes_list.append([end_time_30,(end_time_30 + timedelta(minutes=30)), float('30')])
        return recursive_auto_schedule_30_mins(end_time_30 + timedelta(minutes=30))
    else:
        return __30_minutes_list
    
def recursive_auto_schedule_45_mins(end_time_45):
    if end_time_45 < endtime and endtime != datetime.combine(date.today(),datetime.strptime('003000','%H%M%S').time()):
        __45_minutes_list.append([end_time_45,(end_time_45 + timedelta(minutes=45)), float('45')])
        return recursive_auto_schedule_45_mins(end_time_45 + timedelta(minutes=45))
    else:
        return __45_minutes_list
    
def recursive_auto_schedule_60_mins(end_time_60):
    if end_time_60 < endtime and endtime != datetime.combine(date.today(),datetime.strptime('000000','%H%M%S').time()):
        __60_minutes_list.append([end_time_60,(end_time_60 + timedelta(minutes=60)), float('60')])
        return recursive_auto_schedule_60_mins(end_time_60 + timedelta(minutes=60))
    else:
        return __60_minutes_list

In [0]:
__15_minutes_list_1 = recursive_auto_schedule_15_mins(starttime)
__15_minutes_final_list = doset(__15_minutes_list_1)
cols = ['starttime', 'endtime', 'duration']
df_15_minutes = spark.createDataFrame(__15_minutes_final_list).toDF(*cols)

In [0]:
__30_minutes_list_1 = recursive_auto_schedule_30_mins(starttime)
__30_minutes_final_list = doset(__30_minutes_list_1)
cols = ['starttime', 'endtime', 'duration']
df_30_minutes = spark.createDataFrame(__30_minutes_final_list).toDF(*cols)

In [0]:
__45_minutes_list_1 = recursive_auto_schedule_45_mins(starttime)
__45_minutes_final_list = doset(__45_minutes_list_1)
cols = ['starttime', 'endtime', 'duration']
df_45_minutes = spark.createDataFrame(__45_minutes_final_list).toDF(*cols)

In [0]:
__60_minutes_list_1 = recursive_auto_schedule_60_mins(starttime)
__60_minutes_final_list = doset(__60_minutes_list_1)
cols = ['starttime', 'endtime', 'duration']
df_60_minutes = spark.createDataFrame(__60_minutes_final_list).toDF(*cols)

In [0]:
df_tempHourlyfrequency = df_15_minutes.union(df_30_minutes).union(df_45_minutes).union(df_60_minutes)
df_tempHourlyfrequency.createOrReplaceTempView("tempHourlyfrequency")

## Gettig trigger and wait time from trigger frequency table

In [0]:
df_minutes_wise_triggers = spark.sql(f"""SELECT tr.PipelineName,tr.ParameterName1,tr.ParameterValue1,
                                    tr.ParameterName2,tr.ParameterValue2,tr.ParameterName3,tr.ParameterValue3,
                                    dateadd(minute,try_cast(tr.`Minute` AS INT),c.starttime) AS trigertime
                                    ,cast(current_timestamp() as timestamp) as CurrentTime
                                    ,CASE WHEN SUBSTRING(CAST(c.starttime AS string),12,2)='00' THEN 
                                        datediff(second,cast(current_timestamp() as timestamp),cast(concat('23',':','59') as timestamp))+60+(60*try_cast(tr.`Minute` as INT))+(60*cast((SUBSTRING(CAST(c.starttime AS string),15,2)) AS INT))
                                    ELSE 
                                        datediff(second,cast(current_timestamp() as timestamp),dateadd(minute,try_cast(tr.`Minute` as INT),c.starttime)) 
                                    END as WaitTime
                                    ,tr.ADFType
                                    ,tr.JobID
                                from `{uc_catalog_name}`.`{uc_raw_schema}`.tb_trigger_frequency tr
                                CROSS JOIN tempHourlyfrequency  c
                                where tr.Frequency='Minutes' 
                                    AND tr.`Duration`=cast(c.`Duration` AS FLOAT)
                                    AND tr.ActiveFlag='Y'
                                    AND coalesce(tr.TriggerEndDateTime, '3000-12-31 00:00:00.000') > current_timestamp()
                                    AND tr.ADFType='IngestionADB'""")

In [0]:
df_hourly_wise_triggers = spark.sql(f"""SELECT PipelineName,ParameterName1,ParameterValue1,
                                    ParameterName2,ParameterValue2,ParameterName3,ParameterValue3,
                                    cast(concat(DATE_FORMAT(DATEADD(HOUR,1,current_timestamp()),'HH'),':',`Minute`) as timestamp) AS TrigerTime
                                    ,cast(current_timestamp() as timestamp) as CurrentTime
                                    ,CASE WHEN DATE_FORMAT(DATEADD(HOUR,1,current_timestamp()),'HH')='00' THEN 
                                        datediff(second,cast(current_timestamp() as timestamp),cast(concat('23',':','59') as timestamp))+60+(60*try_cast(`Minute` as INT))
                                    ELSE 
                                        datediff(second,cast(current_timestamp() as timestamp),cast(concat(DATE_FORMAT(DATEADD(HOUR,1,current_timestamp()),'HH'),':',`Minute`) as timestamp)) 
                                    END as WaitTime
                                    ,ADFType
                                    ,JobID
                                from `{uc_catalog_name}`.`{uc_raw_schema}`.tb_trigger_frequency
                                where Frequency='Hourly' 
                                    AND coalesce(TriggerEndDateTime, '3000-12-31 00:00:00.000') > current_timestamp()
                                    AND ((DATE_FORMAT(DATEADD(HOUR,1,current_timestamp()),'HH') - cast(`Hour` as int )) % cast(`Duration` as int )) = 0
                                    AND ActiveFlag='Y'
                                    AND (HOUR(current_timestamp()) between coalesce(`IntervalStart`, 0) and coalesce(`IntervalEnd`, 23))
                                    AND ADFType='IngestionADB'""")

In [0]:
df_regular_interval_triggers = spark.sql(f"""SELECT PipelineName,ParameterName1,ParameterValue1,
                                    ParameterName2,ParameterValue2,ParameterName3,ParameterValue3,
                                    cast(concat(`Hour`,':',`Minute`) as timestamp) AS TrigerTime
                                    ,cast(current_timestamp() as timestamp) as CurrentTime
                                    ,CASE WHEN `Hour`='00' THEN 
                                        datediff(second,cast(current_timestamp() as timestamp),cast(concat('23',':','59') as timestamp))+60+(60*try_cast(`Minute` as INT))
                                    ELSE 
                                        datediff(second,cast(current_timestamp() as timestamp),cast(concat(`Hour`,':',`Minute`) as timestamp)) 
                                    END as WaitTime
                                    ,ADFType
                                    ,JobID
                                from `{uc_catalog_name}`.`{uc_raw_schema}`.tb_trigger_frequency
                                where Frequency='Daily' 
                                    AND coalesce(TriggerEndDateTime, '3000-12-31 00:00:00.000') > current_timestamp()
                                    AND DATEDIFF(day,TriggerStartDateTime,DATEADD(HOUR,1,current_timestamp())) % cast(`Duration` as int) = 0
                                    AND `Hour`=CAST(DATE_FORMAT(DATEADD(HOUR,1,current_timestamp()),'HH') AS INT)
                                    AND ActiveFlag='Y'
                                    AND (DAY(current_timestamp()) between coalesce(IntervalStart, 1) and coalesce(IntervalEnd, 31))
                                    AND ADFType='IngestionADB'""")

In [0]:
df_daily_workday_interval_triggers = spark.sql(f"""SELECT PipelineName,ParameterName1,ParameterValue1,
                                    ParameterName2,ParameterValue2,ParameterName3,ParameterValue3,
                                    cast(concat(`Hour`,':',`Minute`) as timestamp) AS TrigerTime
                                    ,cast(current_timestamp() as timestamp) as CurrentTime
                                    ,CASE WHEN `Hour`='00' THEN 
                                        datediff(second,cast(current_timestamp() as timestamp),cast(concat('23',':','59') as timestamp))+60+(60*try_cast(`Minute` as INT))
                                    ELSE 
                                        datediff(second,cast(current_timestamp() as timestamp),cast(concat(`Hour`,':',`Minute`) as timestamp)) 
                                    END as WaitTime
                                    ,ADFType
                                    ,JobID
                                from `{uc_catalog_name}`.`{uc_raw_schema}`.tb_trigger_frequency
                                where Frequency='Daily_Workday' 
                                    AND coalesce(TriggerEndDateTime, '3000-12-31 00:00:00.000') > current_timestamp()
                                    AND DATEDIFF(day,TriggerStartDateTime,DATEADD(HOUR,1,current_timestamp())) % cast(`Duration` as int) = 0
                                    AND `Hour`=CAST(DATE_FORMAT(DATEADD(HOUR,1,current_timestamp()),'HH') AS INT)
                                    AND ActiveFlag='Y'
                                    AND (DAY(current_timestamp()) between coalesce(IntervalStart, 1) and coalesce(IntervalEnd, 31))
                                    AND date_format(current_timestamp(), 'EEEE') NOT IN ('Saturday','Sunday')
                                    AND ADFType='IngestionADB'""")

In [0]:
df_weekly_triggers = spark.sql(f"""SELECT PipelineName,ParameterName1,ParameterValue1,
                                    ParameterName2,ParameterValue2,ParameterName3,ParameterValue3,
                                    cast(concat(`Hour`,':',`Minute`) as timestamp) AS TrigerTime
                                    ,cast(current_timestamp() as timestamp) as CurrentTime
                                    ,CASE WHEN `Hour`='00' THEN 
                                        datediff(second,cast(current_timestamp() as timestamp),cast(concat('23',':','59') as timestamp))+60+(60*try_cast(`Minute` as INT))
                                    ELSE 
                                        datediff(second,cast(current_timestamp() as timestamp),cast(concat(`Hour`,':',`Minute`) as timestamp)) 
                                    END as WaitTime
                                    ,ADFType
                                    ,JobID
                                from `{uc_catalog_name}`.`{uc_raw_schema}`.tb_trigger_frequency
                                where Frequency='Weekly' 
                                    AND coalesce(TriggerEndDateTime, '3000-12-31 00:00:00.000') > current_timestamp()
                                    AND dayOfWeek = (CASE WHEN 'Hour'='00' THEN date_format(dateadd(day,1,current_timestamp()), 'EEEE') ELSE date_format(current_timestamp(), 'EEEE') END ) 
                                    AND `Hour`=CAST(DATE_FORMAT(DATEADD(HOUR,1,current_timestamp()),'HH') AS INT)
                                    AND ActiveFlag='Y'
                                    AND ADFType='IngestionADB'""")

In [0]:
df_monthly_triggers = spark.sql(f"""SELECT PipelineName,ParameterName1,ParameterValue1,
                                    ParameterName2,ParameterValue2,ParameterName3,ParameterValue3,
                                    cast(concat(`Hour`,':',`Minute`) as timestamp) AS TrigerTime
                                    ,cast(current_timestamp() as timestamp) as CurrentTime
                                    ,CASE WHEN `Hour`='00' THEN 
                                        datediff(second,cast(current_timestamp() as timestamp),cast(concat('23',':','59') as timestamp))+60+(60*try_cast(`Minute` as INT))
                                    ELSE 
                                        datediff(second,cast(current_timestamp() as timestamp),cast(concat(`Hour`,':',`Minute`) as timestamp)) 
                                    END as WaitTime
                                    ,ADFType
                                    ,JobID
                                from `{uc_catalog_name}`.`{uc_raw_schema}`.tb_trigger_frequency
                                where Frequency='Monthly' 
                                    AND coalesce(TriggerEndDateTime, '3000-12-31 00:00:00.000') > current_timestamp()
                                    AND (
                                        (cast(`Duration` as int) = 0  AND (replace(ltrim(replace(dayOfMonth,'0',' ')),' ','0')=CASE WHEN date_format(dateadd(day,1, current_timestamp()),'yyyy-MM-dd') = last_day(current_timestamp()) AND `Hour`='00' then '-1' 
                                        WHEN date_format(current_timestamp(),'yyyy-MM-dd') = last_day(current_timestamp()) THEN '-1' 
                                        WHEN `Hour`='00' THEN day(dateadd(day,1,current_timestamp())) 
                                        ELSE day(current_timestamp()) END ) 
                                        )
                                        --Monthly specific week and day, like 2nd Saturday, 3rd Monday.
                                        OR
                                        (cast(`Duration` as int) > 0 
                                        AND `dayOfWeek` = (
                                            CASE WHEN `Hour`='00' THEN (
                                            CASE WHEN ((dayofweek(dateadd(day,1,current_timestamp())) + 7 - 2) % 7) = 0 THEN 'Monday'
                                                WHEN ((dayofweek(dateadd(day,1,current_timestamp())) + 7 - 2) % 7) = 1 THEN 'Tuesday'
                                                WHEN ((dayofweek(dateadd(day,1,current_timestamp())) + 7 - 2) % 7) = 2 THEN 'Wednesday'
                                                WHEN ((dayofweek(dateadd(day,1,current_timestamp())) + 7 - 2) % 7) = 3 THEN 'Thursday'
                                                WHEN ((dayofweek(dateadd(day,1,current_timestamp())) + 7 - 2) % 7) = 4 THEN 'Friday'
                                                WHEN ((dayofweek(dateadd(day,1,current_timestamp())) + 7 - 2) % 7) = 5 THEN 'Saturday'
                                                WHEN ((dayofweek(dateadd(day,1,current_timestamp())) + 7 - 2) % 7) = 6 THEN 'Sunday'
                                                ELSE 'Invalid Day' END
                                              )
                                            ELSE (
                                                CASE WHEN ((dayofweek(current_timestamp()) + 7 - 2) % 7) = 0 THEN 'Monday'
                                                    WHEN ((dayofweek(current_timestamp()) + 7 - 2) % 7) = 1 THEN 'Tuesday'
                                                    WHEN ((dayofweek(current_timestamp()) + 7 - 2) % 7) = 2 THEN 'Wednesday'
                                                    WHEN ((dayofweek(current_timestamp()) + 7 - 2) % 7) = 3 THEN 'Thursday'
                                                    WHEN ((dayofweek(current_timestamp()) + 7 - 2) % 7) = 4 THEN 'Friday'
                                                    WHEN ((dayofweek(current_timestamp()) + 7 - 2) % 7) = 5 THEN 'Saturday'
                                                    WHEN ((dayofweek(current_timestamp()) + 7 - 2) % 7) = 6 THEN 'Sunday'
                                                    ELSE 'Invalid Day' END
                                            ) END 
                                            ) 
                                        AND cast((day(current_timestamp())-1)/7 AS INT)+1 = cast(`Duration` as int)
                                        )
                                    )
                                    AND `Hour`=CAST(DATE_FORMAT(DATEADD(HOUR,1,current_timestamp()),'HH') AS INT)
                                    AND (month(current_timestamp())) BETWEEN coalesce(IntervalStart, 1) AND coalesce(IntervalEnd, 12)
                                    AND ActiveFlag='Y'
                                    AND ADFType='IngestionADB'""")

In [0]:
df_monthly_workday_triggers = spark.sql(f"""SELECT PipelineName,ParameterName1,ParameterValue1,
                                    ParameterName2,ParameterValue2,ParameterName3,ParameterValue3,
                                    cast(concat(`Hour`,':',`Minute`) as timestamp) AS TrigerTime
                                    ,cast(current_timestamp() as timestamp) as CurrentTime
                                    ,CASE WHEN `Hour`='00' THEN 
                                        datediff(second,cast(current_timestamp() as timestamp),cast(concat('23',':','59') as timestamp))+60+(60*try_cast(`Minute` as INT))
                                    ELSE 
                                        datediff(second,cast(current_timestamp() as timestamp),cast(concat(`Hour`,':',`Minute`) as timestamp)) 
                                    END as WaitTime
                                    ,ADFType
                                    ,JobID
                                from `{uc_catalog_name}`.`{uc_raw_schema}`.tb_trigger_frequency
                                where Frequency='Monthly_Workday' 
                                    AND coalesce(TriggerEndDateTime, '3000-12-31 00:00:00.000') > current_timestamp()
                                    AND (replace(ltrim(replace(dayOfMonth,'0',' ')),' ','0') = CASE WHEN `Hour`='00' THEN ((DATEDIFF(day, DATEADD(month, DATEDIFF(month, CAST(0 AS timestamp), dateadd(day,1,current_timestamp())), CAST(0 AS timestamp)), dateadd(day,1,current_timestamp())) + 1) - (CAST(round(DATEDIFF(DAY, DATEADD(MONTH, DATEDIFF(MONTH, CAST(0 AS timestamp), dateadd(day,1,current_timestamp())), CAST(0 AS timestamp)), dateadd(day,1,current_timestamp()))/7,1) AS INT) * 2) - (CASE WHEN date_format(DATEADD(MONTH, DATEDIFF(MONTH, CAST(0 AS timestamp), dateadd(day,1,current_timestamp())), CAST(0 AS timestamp)), 'EEEE') = 'Sunday' THEN 1 ELSE 0 END) - (CASE WHEN date_format(dateadd(day,1,current_timestamp()), 'EEEE') = 'Saturday' THEN 1 ELSE 0 END)) 
                                    ELSE ((DATEDIFF(day, DATEADD(month, DATEDIFF(month, CAST(0 AS timestamp), current_timestamp()), CAST(0 AS timestamp)), current_timestamp()) + 1) - (CAST(round(DATEDIFF(DAY, DATEADD(MONTH, DATEDIFF(MONTH, CAST(0 AS timestamp), current_timestamp()), CAST(0 AS timestamp)), current_timestamp())/7,1) AS INT) * 2) - (CASE WHEN date_format(DATEADD(MONTH, DATEDIFF(MONTH, CAST(0 AS timestamp), current_timestamp()), CAST(0 AS timestamp)), 'EEEE') = 'Sunday' THEN 1 ELSE 0 END) - (CASE WHEN date_format(current_timestamp(), 'EEEE') = 'Saturday' THEN 1 ELSE 0 END)) END
                                    )
                                    AND `Hour`=CAST(DATE_FORMAT(DATEADD(HOUR,1,current_timestamp()),'HH') AS INT)
                                    AND (month(current_timestamp()) BETWEEN coalesce(IntervalStart, 1) AND coalesce(IntervalEnd, 12))
                                    AND ActiveFlag='Y'
                                    AND ADFType='IngestionADB'""")

In [0]:
df_quarterly_triggers = spark.sql(f"""SELECT PipelineName,ParameterName1,ParameterValue1,
                                    ParameterName2,ParameterValue2,ParameterName3,ParameterValue3,
                                    cast(concat(`Hour`,':',`Minute`) as timestamp) AS TrigerTime
                                    ,cast(current_timestamp() as timestamp) as CurrentTime
                                    ,CASE WHEN `Hour`='00' THEN 
                                        datediff(second,cast(current_timestamp() as timestamp),cast(concat('23',':','59') as timestamp))+60+(60*try_cast(`Minute` as INT))
                                    ELSE 
                                        datediff(second,cast(current_timestamp() as timestamp),cast(concat(`Hour`,':',`Minute`) as timestamp)) 
                                    END as WaitTime
                                    ,ADFType
                                    ,JobID
                                FROM `{uc_catalog_name}`.`{uc_raw_schema}`.tb_trigger_frequency
                                WHERE Frequency='Quarterly' 
                                AND coalesce(TriggerEndDateTime, '3000-12-31 00:00:00.000') > current_timestamp()
                                AND ((month(current_timestamp()) in (1,4,7,10)) OR (month(current_timestamp()) in (12,3,6,9) AND CAST(DATE_FORMAT(current_timestamp(),'HH') AS INT)=23))
                                AND (replace(ltrim(replace(dayOfMonth,'0',' ')),' ','0')=CASE WHEN `Hour`='00' THEN DAY(dateadd(day,1,current_timestamp())) ELSE DAY(current_timestamp()) END ) 
                                AND `Hour`=CAST(DATE_FORMAT(DATEADD(HOUR,1,current_timestamp()),'HH') AS INT)
                                AND ActiveFlag='Y' 
                                AND ADFType='IngestionADB'""")

In [0]:
df_yearly_triggers = spark.sql(f"""SELECT PipelineName,ParameterName1,ParameterValue1,
                                    ParameterName2,ParameterValue2,ParameterName3,ParameterValue3,
                                    cast(concat(`Hour`,':',`Minute`) as timestamp) AS TrigerTime
                                    ,cast(current_timestamp() as timestamp) as CurrentTime
                                    ,CASE WHEN `Hour`='00' THEN 
                                        datediff(second,cast(current_timestamp() as timestamp),cast(concat('23',':','59') as timestamp))+60+(60*try_cast(`Minute` as INT))
                                    ELSE 
                                        datediff(second,cast(current_timestamp() as timestamp),cast(concat(`Hour`,':',`Minute`) as timestamp)) 
                                    END as WaitTime
                                    ,ADFType
                                    ,JobID
                                FROM `{uc_catalog_name}`.`{uc_raw_schema}`.tb_trigger_frequency
                                WHERE Frequency='Yearly' 
                                AND coalesce(TriggerEndDateTime, '3000-12-31 00:00:00.000') > current_timestamp()
                                AND (replace(ltrim(replace(monthOfYear,'0',' ')),' ','0')=month(current_timestamp()) OR (CAST(DATE_FORMAT(current_timestamp(),'HH') AS INT)=23 and replace(ltrim(replace(monthOfYear,'0',' ')),' ','0') = month(current_timestamp())+1))
                                AND (replace(ltrim(replace(dayOfMonth,'0',' ')),' ','0')=CASE WHEN `Hour`='00' THEN DAY(dateadd(day,1,current_timestamp())) ELSE DAY(current_timestamp()) END )
                                AND `Hour`=CAST(DATE_FORMAT(DATEADD(HOUR,1,current_timestamp()),'HH') AS INT)
                                AND ActiveFlag='Y' 
                                AND ADFType='IngestionADB'""")

In [0]:
Total_Triggers = df_minutes_wise_triggers.union(df_hourly_wise_triggers).union(df_regular_interval_triggers).union(df_daily_workday_interval_triggers).union(df_weekly_triggers).union(df_monthly_triggers).union(df_monthly_workday_triggers).union(df_quarterly_triggers).union(df_yearly_triggers)

Total_Triggers_pandas = Total_Triggers.toPandas()
Total_Triggers_Json = Total_Triggers_pandas.to_json(orient='records', date_format='iso',date_unit='ms')
print(Total_Triggers_Json)

In [0]:
dbutils.notebook.exit(Total_Triggers_Json)